## OHLC Minute Data Import, Cleaning and Resampling
In this notebook we process some raw forex data from Dukascopy. The goal is to Import, Clean, Resample and Organize the
Data.

### Imports
We start by importing the **os** and **pandas** modules. We will largely use Pandas for almost everything.

In [18]:
import os
import pandas as pd

### Defining Critical Functions

#### 1. Create an Organized Folder Structure for the Processed OHLC
We use the python **os** module to first of all check to see if the paths to our folders exist. We then create folders for the paths if they don't exist.

In [19]:
def create_folder_if_not_exists(currency_pair, year):
    base_path = 'FXData/CurrencyPairs'

    currency_pair_path = base_path + '/' + currency_pair
    if not os.path.exists(currency_pair_path):
        os.mkdir(currency_pair_path)

    year_path = currency_pair_path + '/' + year
    if not os.path.exists(year_path):
        os.mkdir(year_path)


#### 2. Resample the OHLC to the Defined Interval
Using a dictionary, we first of all map the Dataframe columns to the ones pandas recognises for resampling.

We then use the resample function available to our OHLC Dataframe to aggregate based on our dictionary of column mappings.

We make sure to drop all NaNs and then return a new Dataframe with the required OHLC columns.

In [20]:
def resample_ohlc(ohlc, interval):
    ohlc_dict = {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}
    ohlc = ohlc.resample(interval).agg(ohlc_dict).dropna(how='any')

    cols = ['open', 'high', 'low', 'close', 'volume']
    return ohlc[cols]


#### 3. Parse the Raw Dukascopy Datetime string into an acceptable of Datetime Format
The Raw Dukascopy Datetime is in this format : **01.01.2019 00:00:00.000 GMT+0200** which can not be parsed by Pandas.

Using the **split** function and a single space as our delimiter, we first break the datetime sting into three sections
namely: **Date, Time, Timezone**; which become our List elements inside the **datetime_list**

We use the split function and a **'.'** as our delimiter to break the **Date** which is the first element of
the **datetime_list** into **Day, Month, Year** which make the **date_list**.

Using the delimiter **':'**, we then split the **time** element from the **datetime_list** into **Hour, Minute, Second**
to make the **time_list**.

Finally we return a properly formatted dictionary accepted by Pandas.

In [21]:
def parse_dukascopy_raw_date_time(datetime):
    datetime_list = datetime.split(' ')
    date_list = datetime_list[0].split('.')
    time_list = datetime_list[1].split(':')

    return date_list[2]+"-"+date_list[1]+"-"+date_list[0]+" "+time_list[0]+":"+time_list[1]+":"+time_list[2][:2]

#### 4. Import the OHLC Files and Resample them to typical analysis Timeframes
We use Pandas to read the csv files with OHLC data. We tell pandas to make sure it does the following during the import:

1. Make sure that it parses all dates it finds in the csv file
2. Use the **parse_dukascopy_raw_date_time** custom function we defined to parse the dates
3. Use the first row of the csv file as the column header and select the columns to import
4. Use the 0th column index as Dataframe index
5. Define all values to be set as NaN
6. The delimiter of the csv file.

After the import is successful, we define a list of intervals for resampling.

We also check to see if an organised folder structure exists for the currency pair and year.

Now for each interval we resample the OHLC Minute data and then Save the returned Dataframe in a csv at the specified
path

In [22]:
def import_from_dukascopy_ohlc(path, currency_pair, year, delimiter=','):
    ohlc = pd.read_csv(path + '.csv',
                       parse_dates=True,
                       date_parser=parse_dukascopy_raw_date_time,
                       usecols=['datetime', 'open', 'high', 'low', 'close', 'volume'],
                       index_col=0,
                       na_values=['nan'],
                       delimiter=delimiter)

    intervals = ['1Min', '5Min', '15Min', '30Min', '1H', '2H', '4H', '6H', '8H', '1D', '1W', '1M']
    create_folder_if_not_exists(currency_pair, year)

    for interval in intervals:
        ohlc = resample_ohlc(ohlc, interval)
        ohlc = ohlc[(ohlc.open != ohlc.high) | (ohlc.high != ohlc.low) | (ohlc.low != ohlc.close)].round(5)
        ohlc.to_csv('FXData/CurrencyPairs/'+str(currency_pair)+'/'+str(year)+'/'+str(interval)+'.csv')



### Define Configurations

The **years** list defines the years which your files have data for. Make sure the file names take the following
format : CurrencyPairM1Year e.g GBPUSDM12020

In [23]:
years = ['2018', '2019']

The **currency_pairs** list defines the currency pairs which your files have data for.

In [24]:
currency_pairs = [
    'EURUSD'
]

### Process the Files
For each currecny pair you want to go inside each configured year and create the filename then import it using the
**import_from_dukascopy_ohlc** function.

In [38]:
for currency_pair in currency_pairs:
    for year in years:
        path = currency_pair + 'M1' + year
        print('Importing from '+path+'.csv')
        import_from_dukascopy_ohlc(path=path, currency_pair=currency_pair, year=year, delimiter=',')
        print('Done Importing')
        print('####################################')

Importing from EURUSDM12018.csv
Done Importing
####################################
Importing from EURUSDM12019.csv
Done Importing
####################################
